In [43]:
import pandas as pd
import numpy as np
import os
import dotenv
from dotenv import load_dotenv
from common.PortConnect import Port_Connect

In [44]:
load_dotenv()
api_key = os.getenv("API_KEY")

fmp = Port_Connect(api_key=api_key)
apple = fmp.historical_closing_price('SYK')
apple



/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)


,symbol,close
date,,
1979-05-02,SYK,0.083335
1979-05-03,SYK,0.086421
1979-05-04,SYK,0.081792
1979-05-07,SYK,0.077162
1979-05-08,SYK,0.077162
...,...,...
2025-06-03,SYK,381.010000
2025-06-04,SYK,382.550000
2025-06-05,SYK,383.100000


## Calculate Key Metric Statistics for a holding

In [45]:
from common.Portfolio import Portfolio_Stats



In [46]:
port = Portfolio_Stats()

In [47]:
change = apple['close'].pct_change()
change

date
1979-05-02         NaN
1979-05-03    0.037031
1979-05-04   -0.053563
1979-05-07   -0.056607
1979-05-08    0.000000
                ...   
2025-06-03    0.000394
2025-06-04    0.004042
2025-06-05    0.001438
2025-06-06    0.002062
2025-06-09   -0.004194
Name: close, Length: 11622, dtype: float64

In [48]:
type(change)

pandas.core.series.Series

In [49]:
filtered_change = change.loc['2010-01-01':'2025-12-31']

In [50]:
port.annualize_rets(filtered_change,252)

0.14061427116880565

In [51]:
port.annualize_vol(filtered_change,252)

0.24231458918568177

In [52]:
max_duration = port.drawdown_duration_from_returns(filtered_change)
max_time = max_duration.max()
max_time

Timedelta('724 days 00:00:00')

In [60]:
tickers = ['SYK','ISRG','BLK','BSX','HD']

prices = fmp.get_closing_prices(tickers,from_date='2010-01-01')
prices

/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: T

symbol,BLK,BSX,HD,ISRG,SYK
date,,,,,
2010-01-04,238.58,9.01,28.67,34.31,52.16
2010-01-05,239.61,9.04,28.88,34.15,52.65
2010-01-06,234.67,9.16,28.78,34.65,53.46
2010-01-07,237.25,9.09,29.12,34.52,55.25
2010-01-08,238.92,9.00,28.98,34.33,55.42
...,...,...,...,...,...
2025-06-03,982.13,103.10,373.08,551.85,381.01
2025-06-04,988.97,102.43,372.40,557.95,382.55
2025-06-05,985.46,102.83,369.28,558.06,383.10


In [62]:
dfs = []
for ticker in prices.columns:
    change = prices[ticker].pct_change()
    summary = port.summary_stats(change)
    dfs.append(summary)


pd.concat(dfs)


,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown
BLK,0.096692,0.277008,0.200320,0.323284,0.051313,9.891567,0.025498,0.040000,-0.453314
BSX,0.169137,0.283951,0.208881,0.646801,-0.136616,9.266735,0.027064,0.041629,-0.483368
HD,0.179608,0.233039,0.178084,0.815758,-0.618597,17.443548,0.021576,0.033392,-0.384058
ISRG,0.193903,0.320829,0.234652,0.678241,0.266761,12.652051,0.026836,0.045976,-0.499042
SYK,0.138032,0.242183,0.184071,0.569902,-0.109296,13.387988,0.021733,0.035554,-0.438028


In [ ]:
port.summary_stats(filtered_change)

,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown
close,0.140614,0.242315,0.184071,0.58348,-0.107775,13.362683,0.021738,0.035554,-0.438028


In [ ]:
port.cvar_historic(filtered_change)

0.03555434142407394

In [ ]:
port.drawdown(filtered_change)

,Wealth,Previous Peak,Drawdown
date,,,
2010-01-04,1035.537026,1035.537026,0.000000
2010-01-05,1045.265039,1045.265039,0.000000
2010-01-06,1061.346039,1061.346039,0.000000
2010-01-07,1096.883065,1096.883065,0.000000
2010-01-08,1100.258090,1100.258090,0.000000
...,...,...,...
2025-06-03,7564.224737,7939.249553,-0.047237
2025-06-04,7594.798491,7939.249553,-0.043386
2025-06-05,7605.717689,7939.249553,-0.042011


In [ ]:
filtered_change

date
2010-01-04    0.035537
2010-01-05    0.009394
2010-01-06    0.015385
2010-01-07    0.033483
2010-01-08    0.003077
                ...   
2025-06-03    0.000394
2025-06-04    0.004042
2025-06-05    0.001438
2025-06-06    0.002062
2025-06-09   -0.004194
Name: close, Length: 3882, dtype: float64